# Spring Boot Actuator
This notebook contains my notes from the *Spring Boot Actuator* talk by Craig Walls at UberConf 2019. In this talk, he gave a nice tour of the actuator, showing how to use and extend it.

Much of the session was live-coding. It can be found at https://github.com/habuma/nfjs-samples in the *uberconf-2019* folder.

### Adding the Actuator
Adding the `spring-boot-starter-actuator` starter will automatically load and configure the actuator in your application.

All endpoints are under `/actuator`. As a result you can easily secure all the actuator endpoints using something like Spring Security. At FS we use a different port that has network firewalls protecting it, but authorizing the path is another solution.

Hitting the root path `/actuator` will give you some HAL hypmermedia links that give you the available actions in the actuator. What you see will depend on what endpoints you have turned on. By default, only `/actuator/info` and `/actuator/health` are turned on.

### Info Endpoint
By default, the `/actuator/info` endpoint gives you nothing. You can add to it in several ways:

First, you can add to it by setting static properties from your `application.properties` file:

```
info.support.email=support@example.xom
info.support.phone=555-555-5555
```

Any property prefixed with `info.` will be placed in the info endpoint.

In addition to static configuration, you can do the usual dynamic property replacement in the configuration file too:

```
info.serverPort: ${local.server.port}
```

This is still pretty limited because you can't use the property files to pull values from somewhere like a database. You can obtain these more dynamic values by adding an `InfoContributor`. The below is a (somewhat silly) example of providing a value for the `/actuator/info` endpoint:

In [4]:
%maven org.springframework:spring-beans:5.1.8.RELEASE
%maven org.springframework.boot:spring-boot-actuator:2.1.5.RELEASE

import org.springframework.stereotype.Component;
import org.springframework.boot.actuate.info.InfoContributor;
import org.springframework.boot.actuate.info.Info.Builder;

@Component
public class RandomInfoContributor implements InfoContributor {

    @Override
    public void contribute(Builder builder) {
        double random = Math.random();
        builder.withDetail("random", random);
    }

}

You can add build information from the Maven POM file by adding the `build-info` goal to the spring-boot-plugin executions.

You can also add Git commit information using the git-commit-id-plugin. You can choose 'full' or 'simple' in the actuator config.

### Health Endpoint
Available at `/actuator/health`. The health endpoint isn't just a simple heartbeat up/down. It reports the aggregate health of all the health indicators in the application as well.

All indicators need to report UP for the health endpoint to report UP.

A lot of health indicators are available and configured out-of-the-box. For example, when you use Spring Boot to add a DataSource, there will be a corresponding health indicator for it. There is always a disk space indicator configured as well.

By default the endpoint just shows the aggregate UP/DOWN. You can set `managment.endpoint.health.show-details=always` to make the endpoint show all the health indicator statuses in addition to the aggregate.

You can also easily implement your own health indicators. Here's a really silly example that reports DOWN about half the time using a random number generator:

In [9]:
import org.springframework.boot.actuate.health.HealthIndicator;
import org.springframework.boot.actuate.health.Health;

@Component
public class RandomHealthIndicator implements HealthIndicator {
    
    @Override
    public Health health() {
        double random = Math.random();
        if(random > 0.5) {
            return Health
                .up()
                .withDetail("reason", "Number is > 0.5")
                .build();
        }
        
        return Health
            .down()
            .withDetail("reason", "Number is <= 0.5")
            .build();
    }

}

You can also see individual health indicator information at `/actuator/health/{indicator}`

**Question**: Are health indicator statuses periodically checked on a schedule, or are they only checked when you call  the actuator health endpoint? I'm guessing it's only when called.

### Enabling Other Actuator Endpoints
You can enable actuator endpoints using the following config property:

```
management.endpoints.web.exposure.include=*
```

The above example enables them all. You can also just use a comma-separated list to enable only some of them. There is also a corresponding `exclude` property that allows you to exclude actuator endpoints.

### Audit Events Endpoint
Available at `/actuator/auditevents`. Craig said this is useful when you are using Spring Security. This endpoint is registered as a listener for the audit events generated by Spring Security. This means things like failed logins will show up here. 

### Beans Endpoint
Available at `/actuator/beans`. This contains information for all the beans that are registered and available in your application context. It's a lot of information, but tells you exactly what Spring has instantiated.

### Caches Endpoint
This endpoint is useful if you are using a cache, such as Redis or Caffeine. If you have configured it using Spring Boot then you can see information about the cache, and can even do actions like purging it (I think).

### Conditions Endpoint
This endpoint gives information about the conditions considered during auto-configuration (missing bean, etc.). It will tell you the conditions that succeeded, failed, etc. Craig said this can be very helpful in figuring out why Spring Boot did what it did during startup and autoconfiguration.

### Env Endpoint
This endpoint was covered more in Craig's *Extreme Spring Boot* session. It contains information about the PropertySources in the Spring Environment.

In addition to viewing the top-level Environment information, you can use `/env/{propertyName}` to see which PropertySources (if any) provide a specific property.

### Loggers Endpoint
This endpoint shows configured and effective logging level for every package in the application. 

The *configured* level(if present) shows levels that were explicitly configured somewhere. For example, if you see *configured=Info*, it means that someone explicitly set that package to Info-level logging somewhere. *Effective* level means the package was either explicitly configured **or** inherited from a parent package.

You can send a POST to these endpoints to change your logging level on packages.

### Scheduled Tasks Endpoint
This endpoint shows the scheduled tasks in your application.

### Http Trace Endpoint
This endpoint remembers information about the last *n* HTTP requests. It's a small number (like 100), so I'm not sure what practical utility this has for applications of any sufficient scale. Craig didn't say much about this endpoint.

### Mappings
This endpoint shows all the Spring MVC request mapping information for your application. This can be helpful to figure out why your routing isn't working the way you think it should.

### JMX in Actuator
By default, the actuator exposes *all* its endpoints over JMX. You have to explicitly turn this off if you don't want it.

### Creating your own actuator endpoint
It's pretty easy to make your own actuator endpoints:

In [12]:
import org.springframework.boot.actuate.endpoint.annotation.Endpoint;
import org.springframework.boot.actuate.endpoint.annotation.ReadOperation;

@Component
@Endpoint(id="uberconf", enableByDefault=true)
public class UberConfEndpoint {

    @ReadOperation
    public String getMagicPhrase() {
        return "Some Magic Phrase";
    }
    
}

There is a corresponding `@WriteOperation` annotation that you can use when creating actuator endpoints.

### Spring Boot Admin UI
CodeCentric makes a Spring Boot Admin console that your actuators can hook into: https://github.com/codecentric/spring-boot-admin. This client is *NOT* made by the Spring team; it is from a different organization.

You run the console as a separate application, then you have your Spring Boot application add a client library to talk to the console. This client will report your actuator information to the admin console.

### Metrics Endpoint
This endpoint exposes metrics of all sorts in your Spring application. There are a lot of metrics supported out-of-the-box with Spring Boot. 

Micrometer is the library used for metrics in Spring Boot. This allows you to not only use the actuator for metrics, but also send those same metrics to other logging/metrics platforms like NewRelic.

You can also add your own metrics very easily. Just autowire a @MeterRegistry into your class and use it to report metrics:

In [17]:
%maven io.micrometer:micrometer-core:1.2.0

import org.springframework.beans.factory.annotation.Autowired;
import io.micrometer.core.instrument.MeterRegistry;

@Component
@Endpoint(id="uberconf", enableByDefault=true)
public class MyMetricsExample {

    private final MeterRegistry registry;
    
    @Autowired
    public MyMetricsExample(MeterRegistry registry) {
        this.registry = registry;
    }
    
    public void doSomething() {
        // do something here
        
        // increment a counter metric
        registry.counter("uberconf", "hello", "world").increment();
    }
    
}

### Takeaways
* I should look at what health indicators are in our applications, as well as what indicators we could add. 
    * The aggregate health status would be a good thing to hook up to an Apica check
* Spring Cloud Config Server has its own health indicator in the config client. Perhaps this is another reason to use Spring Cloud Config?
* I should check in AppDynamics to see if JMX beans from the actuator are there.
* Much of the dashboards and alerts that we put in Splunk for our applications can be done more efficiently using metrics instead of logging.